In [1]:
'''
Created on October 31st 2017
@author: Juan Manuel Acevedo Valle
''' 
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
import numpy as np
import h5py, os, sys, random
from scipy.stats import gaussian_kde
from scipy import stats
from scipy.stats.distributions import norm    

In [2]:
%matplotlib inline
import os
import itertools
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy.stats import pearsonr, shapiro, anderson
from scipy.spatial import ConvexHull

colors_ = ['firebrick', 'darkgreen', 'navy', 'm', 'deepskyblue', 'goldenrod', 'black', 'grey']

from exploration.data.data import load_sim_h5_v2 as load_sim_h5
from exploration.data.PlotTools import *
from results_utils import *
from exploration.algorithm.utils.logging import read_config_log

In [3]:
# directory =  'experiment_1_epirob/'
directory = 'diva_experiment_thesis_chap_5_toshi/'

case = 'simple' #'simple'
data_files = os.listdir(directory)
data_files = [file_ for file_ in os.listdir(directory) if case in file_]


n_samples = 102000
# Group by:
mode_ops = ['autonomous']
# type_ops = ['proprio','simple']
type_ops = [case]

groups = itertools.product(mode_ops, type_ops)

for data_file in (directory + d_f for d_f in data_files if 'sim_data.h5' in d_f):
    data_file = data_file
    conf_file = data_file.replace('sim_data.h5', 'conf.txt')
    conf = read_config_log(conf_file)
#     conf = {}
#     with open(conf_file) as f:
#         for line in f:
#             line = line.replace('\n', '')
#             (key, val) = line.split(': ')
#             conf[key] = val
    data,foo = load_sim_h5(data_file)
    try:
        concat_action = np.append(concat_action, data.action.data.as_matrix(), axis=0)
#         print "Appending"
    except:
        concat_action = data.action.data.as_matrix()
#         print "First set"

# print('Processing ' + group)
print("The shape of the data for PCA is:")
print(concat_action.shape)
pca = PCA(n_components = 26)      
pca.fit(concat_action)
print("Variance contribution per per principal axes: ")
print(str(pca.explained_variance_))    
print("% Variance contribution per per principal axes: ")
print( str(pca.explained_variance_ratio_))    
print("Principal directions [n_components_n_features]:")
print(pca.components_)


The shape of the data for PCA is:
(612000L, 26L)
Variance contribution per per principal axes: 
[ 15.87799698  13.45018059  11.62343739   8.19217167   7.83896656
   5.61619007   5.48297779   5.16248079   4.25741399   3.17024982
   2.90490276   2.44374001   2.31537491   2.04737163   1.75436493
   1.61720143   1.57504397   1.35634556   1.15820253   1.10528518
   0.1934564    0.1572727    0.09957046   0.08173695   0.06573387
   0.0568124 ]
% Variance contribution per per principal axes: 
[ 0.15941047  0.1350359   0.11669593  0.08224702  0.07870094  0.05638491
  0.0550475   0.0518298   0.0427432   0.03182839  0.02916438  0.02453444
  0.02324569  0.02055502  0.01761331  0.01623623  0.01581298  0.01361731
  0.01162802  0.01109674  0.00194225  0.00157897  0.00099966  0.00082062
  0.00065995  0.00057038]
Principal directions [n_components_n_features]:
[[  1.13550698e-01  -4.33574962e-01  -2.08366461e-01  -2.57059737e-01
   -2.71638670e-01   1.42550307e-01  -3.56434545e-01  -2.86784793e-01
    

In [4]:
sum(pca.explained_variance_ratio_[:17])

0.95708610701193775

In [14]:
import h5py, random
from scipy.stats import gaussian_kde
from scipy.stats.distributions import norm    

def plotPDFx_y(axes, data, color):
    x_grid = np.linspace(-2.0, 2., 500)
    y_grid = np.linspace(-2.0, 2., 500)
    
    sensor_data = pca.transform(data)
    
    # The grid we'll use for plotting

    #------ kde = gaussian_kde(x, bw_method=bandwidth / x.std(ddof=1), **kwargs) 
    # Note that scipy weights its bandwidth by the covariance of the
    # input data.  To make the results comparable to the other methods,
    # we divide the bandwidth by the sample standard deviation here.

    # pdf = gaussian_kde(sensor_data1[:,0], bw_method=0.02/  sensor_data1[:,0].std(ddof=1)).evaluate(x_grid)
    kde_x = gaussian_kde(sensor_data[:,0])
    pdf_x = kde_x.evaluate(x_grid)
    axes[0].plot(x_grid, pdf_x, color=color, alpha=0.4, lw=3.5)
    axes[0].set_xlim(-2, 2)

    
    # pdf = gaussian_kde(sensor_data1[:,1], bw_method=0.02 / sensor_data1[:,1].std(ddof=1)).evaluate(y_grid)
    kde_y = gaussian_kde(sensor_data[:,1])
    pdf_y = kde_y.evaluate(y_grid)
    axes[1].plot(y_grid, pdf_y, color=color, alpha=0.4, lw=3.5)
    axes[1].set_xlim(-2, 2)
                 
    tmp_dict = {'KDEx_': kde_x, 'KDEy_': kde_y, 'PDFx_': pdf_x, 'PDFy_': pdf_y }
    return tmp_dict

In [ ]:
dist_used = 'euclidean'
sensor_data = None

matplotlib.rcParams['figure.figsize'] = (12.0, 5.5)
matplotlib.rcParams.update({'font.size': 18})

fig, axes = plt.subplots(1,2)
    
data_files = [directory + file_ for file_ in os.listdir(directory)]
for data_file in (d_f for d_f in data_files if 'sim_data.h5' in d_f):
#     data_file = data_file
    conf_file = data_file.replace('sim_data.h5', 'conf.txt')
    conf = read_config_log(conf_file)
    
#     with open(conf_file) as f:
#         for line in f:
#             line = line.replace('\n', '')
#             line = line.replace('\r', '')
#             (key_, val) = line.split(': ')
#             conf[key_] = val
    data,foo = load_sim_h5(data_file)
    if conf['type']=='simple':
        try:
            concat_sensor_s = np.append(concat_sensor_s, data.sensor.data.as_matrix(), axis=0)
        except:
            concat_sensor_s = data.sensor.data.as_matrix()
    elif conf['type']=='proprio':
        try:
            concat_sensor_p = np.append(concat_sensor_p, data.sensor.data.as_matrix(), axis=0)
        except:
            concat_sensor_p = data.sensor.data.as_matrix()    
plotPDFx_y(axes, concat_sensor_s[:,[0,1,3,4]], colors_[0])
#del concat_sensor_s
plotPDFx_y(axes, concat_sensor_p[:,[0,1,3,4]], colors_[1])

plt.sca(axes[0])
plt.xlabel('$PCA_1$')
plt.sca(axes[1])
plt.xlabel('$PCA_2$')

lgd = ['simple', 'somesthetic']
lgd = plt.legend(lgd,ncol=5,loc='lower right', fontsize=18, bbox_to_anchor=(.4, -0.2),
          fancybox=True, shadow=True)

In [7]:
# fig.savefig('/home/yumilceh/Dropbox/Aplicaciones/ShareLaTeX/PhD_Thesis/chapters/chapter5/images/PCA.eps',\
#                format='eps', dpi=1000,bbox_extra_artists=(lgd,), bbox_inches='tight')

In [13]:
sum(pca.explained_variance_ratio_[:17])

0.94990349846786704